<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Image_Segmentaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 137.67 MiB | 32.75 MiB/s, done.
Resolving deltas: 100% (558/558), done.


In [3]:
%cd Mask_RCNN

/content/Mask_RCNN
